<a href="https://colab.research.google.com/github/chenzhonghuangjun/Bike-sharing/blob/main/Repayment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Merge datasets##

In [3]:
import pandas as pd
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths to the datasets
file_names = [
    'Assets_2020.csv',
    'Baseline_survey_data.csv',
    'BRAC_arrears_data.csv',
    'BRAC_default_data.csv',
    'BRAC_eligibles_loanV_data.csv',
    'BRAC_voucher_data.csv',
    'Endline_survey_data.csv',
    'Midline_survey_data.csv',
    'Rainfall.csv'
]

# Load all datasets into a dictionary of DataFrames
datasets = {name: pd.read_csv(f'/content/drive/My Drive/Rep/Test/{name}') for name in file_names}



Mounted at /content/drive


In [5]:
# Load the datasets from the previously defined dictionary
baseline = datasets['Baseline_survey_data.csv']
midline = datasets['Midline_survey_data.csv']
endline = datasets['Endline_survey_data.csv']
brac_loans = datasets['BRAC_eligibles_loanV_data.csv']

# Append baseline, midline, and endline datasets
# Add a 'followup' column to distinguish between the datasets
baseline['followup'] = 'baseline'
midline['followup'] = 'midline'
endline['followup'] = 'endline'

# Combine the datasets by appending them
panel_temp = pd.concat([baseline, midline, endline], ignore_index=True)

# Sort the combined dataset by 'id' and 'followup'
panel_temp.sort_values(by=['id', 'followup'], inplace=True)

# Ensure the 'followup' column in both DataFrames is of the same type (string)
panel_temp['followup'] = panel_temp['followup'].astype(str)
brac_loans['followup'] = brac_loans['followup'].astype(str)

# Merge with the BRAC loans data on 'id' and 'followup'
merged_data = pd.merge(panel_temp, brac_loans, on=['id', 'followup'], how='left')

# Optionally, check the merge status (though pandas does not create a _merge column by default)
# This would have been done with _merge in Stata, but pandas requires specifying it
merge_status = merged_data.isna().sum()
print("Merge status by column (NaN counts):")
print(merge_status)

# Filter the dataset for 'dabi' or 'progoti' borrowers
filtered_data = merged_data[(merged_data['dabi'] == 1) | (merged_data['progoti'] == 1)]

# Drop any columns that you don't need (like the merge indicator if manually added)
# For example, if you had a column _merge, you'd drop it here
# filtered_data.drop(columns=['_merge'], inplace=True)

# Save the final dataset to a CSV file
filtered_data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp.csv', index=False)

print("Data processing complete. The final dataset has been saved to 'panel_temp.csv'.")


Merge status by column (NaN counts):
id                       0
region                   1
identifier_branch        1
brac_loanee_vo_no    15561
followup                 0
                     ...  
wage_inc_resp           11
int_done              6245
BRACLoanD            18844
BRACLoanValue        18844
Default_y            18844
Length: 157, dtype: int64
Data processing complete. The final dataset has been saved to 'panel_temp.csv'.


In [6]:
# Load the Rainfall dataset
rainfall = datasets['Rainfall.csv']

# Generate 'jan_dec' variables for the years 2013 and 2014
for y in range(2013, 2015):
    rainfall[f'jan_dec{y}'] = (
        rainfall[f'Rain{y}1'] + rainfall[f'Rain{y}2'] + rainfall[f'Rain{y}3'] +
        rainfall[f'Rain{y}4'] + rainfall[f'Rain{y}5'] + rainfall[f'Rain{y}6'] +
        rainfall[f'Rain{y}7'] + rainfall[f'Rain{y}8'] + rainfall[f'Rain{y}9'] +
        rainfall[f'Rain{y}10'] + rainfall[f'Rain{y}11'] + rainfall[f'Rain{y}12']
    )

# Generate 'dec_may' variables for the periods from 1983-1990 and 1992-2016
for y in range(1983, 1991):
    x = y + 1
    rainfall[f'dec_may{x}'] = (
        rainfall[f'dec{y}'] + rainfall[f'jan{x}'] + rainfall[f'feb{x}'] +
        rainfall[f'mar{x}'] + rainfall[f'apr{x}'] + rainfall[f'may{x}']
    )

# Handle the special case for the year 1992 (rainfall is missing for February 1992)
rainfall['dec_may1992'] = (
    rainfall['dec1991'] + rainfall['jan1992'] + rainfall['mar1992'] +
    rainfall['apr1992'] + rainfall['may1992']
)

# Continue generating 'dec_may' variables from 1992-2016
for y in range(1992, 2017):
    x = y + 1
    rainfall[f'dec_may{x}'] = (
        rainfall[f'dec{y}'] + rainfall[f'jan{x}'] + rainfall[f'feb{x}'] +
        rainfall[f'mar{x}'] + rainfall[f'apr{x}'] + rainfall[f'may{x}']
    )

# Save the cleaned and aggregated dataset
rainfall.to_csv('/content/drive/My Drive/Rep/Test/Rainfall_cleaned.csv', index=False)

print("Rainfall data processing complete. The cleaned dataset has been saved as 'Rainfall_cleaned.csv'.")


Rainfall data processing complete. The cleaned dataset has been saved as 'Rainfall_cleaned.csv'.


In [7]:
# Calculate historical means and standard deviations for December to May (1984-2015, 1984-2016, 1984-2017)
years = list(range(1984, 2016 + 1))  # All years from 1984 to 2016

# Mean and SD for 2016 (using data up to 2015)
rainfall['dec_maymean15'] = rainfall[[f'dec_may{year}' for year in years[:-1]]].mean(axis=1)
rainfall['dec_maysd15'] = rainfall[[f'dec_may{year}' for year in years[:-1]]].std(ddof=0, axis=1)

# Mean and SD for 2017 (using data up to 2016)
rainfall['dec_maymean16'] = rainfall[[f'dec_may{year}' for year in years]].mean(axis=1)
rainfall['dec_maysd16'] = rainfall[[f'dec_may{year}' for year in years]].std(ddof=0, axis=1)

# Mean and SD for 2017 (using data up to 2017)
years.append(2017)
rainfall['dec_maymean17'] = rainfall[[f'dec_may{year}' for year in years]].mean(axis=1)
rainfall['dec_maysd17'] = rainfall[[f'dec_may{year}' for year in years]].std(ddof=0, axis=1)

# Standardize the rainfall for 2016 and 2017
rainfall['SD_dec_may2016'] = (rainfall['dec_may2016'] - rainfall['dec_maymean15']) / rainfall['dec_maysd15']
rainfall['SD_dec_may2017'] = (rainfall['dec_may2017'] - rainfall['dec_maymean16']) / rainfall['dec_maysd16']

# Save the updated rainfall dataset
rainfall.to_csv('/content/drive/My Drive/Rep/Test/Rainfall_cleaned_with_means.csv', index=False)

print("Rainfall data processing complete. The dataset with historical means and standard deviations has been saved as 'Rainfall_cleaned_with_means.csv'.")


Rainfall data processing complete. The dataset with historical means and standard deviations has been saved as 'Rainfall_cleaned_with_means.csv'.


In [8]:
# Calculate a 1-standard deviation shock based on the continuous measure
rainfall['SD_dec_may2016_1'] = (rainfall['SD_dec_may2016'] > 1).astype(int)
rainfall['SD_dec_may2017_1'] = (rainfall['SD_dec_may2017'] > 1).astype(int)

# Save the updated rainfall dataset with the new binary columns
rainfall.to_csv('/content/drive/My Drive/Rep/Test/Rainfall_with_SD_shocks.csv', index=False)

print("Rainfall data processing complete. The dataset with 1-standard deviation shocks has been saved as 'Rainfall_with_SD_shocks.csv'.")


Rainfall data processing complete. The dataset with 1-standard deviation shocks has been saved as 'Rainfall_with_SD_shocks.csv'.


In [9]:
# Keep only the relevant columns
rainfall_relevant = rainfall[['identifier_branch', 'jan_dec2013', 'jan_dec2014', 'SD_dec_may2016_1', 'SD_dec_may2017_1']]

# Sort the DataFrame by 'identifier_branch'
rainfall_relevant = rainfall_relevant.sort_values(by='identifier_branch')

# Save the filtered and sorted dataset
rainfall_relevant.to_csv('/content/drive/My Drive/Rep/Test/Rainfall_relevant.csv', index=False)

print("Relevant rainfall data saved as 'Rainfall_relevant.csv'.")


Relevant rainfall data saved as 'Rainfall_relevant.csv'.


In [10]:
# Merge the rainfall data with the survey data using 'identifier_branch'
merged_data = pd.merge(panel_temp, rainfall_relevant, on='identifier_branch', how='outer')

# Display merge status if necessary
merge_status = merged_data.isna().sum()
print("Merge status by column (NaN counts):")
print(merge_status)

# Filter rows where (dabi == 1) or (progoti == 1)
filtered_data = merged_data[(merged_data['dabi'] == 1) | (merged_data['progoti'] == 1)]

# Optionally, drop the '_merge' column if it was created manually
# However, in pandas, the '_merge' column isn't created automatically unless specified
# For example, if '_merge' was manually created or exists:
# filtered_data.drop(columns=['_merge'], inplace=True)

# Save the final merged and filtered dataset
filtered_data.to_csv('/content/drive/My Drive/Rep/Test/merged_survey_rainfall.csv', index=False)

print("Survey and rainfall data merged successfully and saved as 'merged_survey_rainfall.csv'.")

Merge status by column (NaN counts):
id                       0
region                   1
identifier_branch        1
brac_loanee_vo_no    15561
followup                 0
                     ...  
int_done              6245
jan_dec2013              1
jan_dec2014              1
SD_dec_may2016_1         1
SD_dec_may2017_1         1
Length: 158, dtype: int64
Survey and rainfall data merged successfully and saved as 'merged_survey_rainfall.csv'.


In [13]:
import numpy as np
merged_data = pd.read_csv('/content/drive/My Drive/Rep/Test/merged_survey_rainfall.csv')

# Initialize the new column `SD_dec_may_1` to NaN (or any default value)
merged_data['SD_dec_may_1'] = np.nan

# Assign values based on the followup column
merged_data.loc[merged_data['followup'] == 1, 'SD_dec_may_1'] = merged_data['SD_dec_may2016_1']
merged_data.loc[merged_data['followup'] == 2, 'SD_dec_may_1'] = merged_data['SD_dec_may2017_1']

# Save the updated DataFrame
merged_data.to_csv('/content/drive/My Drive/Rep/Test/merged_survey_rainfall_with_SD.csv', index=False)

print("Rainfall data matched with survey data and saved as 'merged_survey_rainfall_with_SD.csv'.")


Rainfall data matched with survey data and saved as 'merged_survey_rainfall_with_SD.csv'.


In [14]:
# Ensure the data is loaded
merged_data = pd.read_csv('/content/drive/My Drive/Rep/Test/merged_survey_rainfall_with_SD.csv')

# Create quartiles for jan_dec2013 and jan_dec2014 for dabi group
merged_data['jan_dec2013_4'] = np.nan
merged_data['jan_dec2014_4'] = np.nan
merged_data['jan_dec2013_4_p'] = np.nan
merged_data['jan_dec2014_4_p'] = np.nan

# Apply quartile computation only for dabi == 1
dabi_mask = merged_data['dabi'] == 1
merged_data.loc[dabi_mask, 'jan_dec2013_4'] = pd.qcut(merged_data.loc[dabi_mask, 'jan_dec2013'], q=4, labels=False, duplicates='drop') + 1
merged_data.loc[dabi_mask, 'jan_dec2014_4'] = pd.qcut(merged_data.loc[dabi_mask, 'jan_dec2014'], q=4, labels=False, duplicates='drop') + 1

# Apply quartile computation only for progoti == 1
progoti_mask = merged_data['progoti'] == 1
merged_data.loc[progoti_mask, 'jan_dec2013_4_p'] = pd.qcut(merged_data.loc[progoti_mask, 'jan_dec2013'], q=4, labels=False, duplicates='drop') + 1
merged_data.loc[progoti_mask, 'jan_dec2014_4_p'] = pd.qcut(merged_data.loc[progoti_mask, 'jan_dec2014'], q=4, labels=False, duplicates='drop') + 1

# Create dummy variables based on the quartile columns
dum1 = pd.get_dummies(merged_data['jan_dec2014_4'], prefix='dum1')
dum2 = pd.get_dummies(merged_data['jan_dec2013_4'], prefix='dum2')
dum1_p = pd.get_dummies(merged_data['jan_dec2014_4_p'], prefix='dum1_p')
dum2_p = pd.get_dummies(merged_data['jan_dec2013_4_p'], prefix='dum2_p')

# Concatenate the dummy variables with the original DataFrame
merged_data = pd.concat([merged_data, dum1, dum2, dum1_p, dum2_p], axis=1)

# Save the updated DataFrame
merged_data.to_csv('/content/drive/My Drive/Rep/Test/merged_survey_rainfall_with_quartiles.csv', index=False)

print("Quartiles and dummy variables created and saved as 'merged_survey_rainfall_with_quartiles.csv'.")


Quartiles and dummy variables created and saved as 'merged_survey_rainfall_with_quartiles.csv'.


In [15]:
# Load the data if it's not already loaded
merged_data = pd.read_csv('/content/drive/My Drive/Rep/Test/merged_survey_rainfall_with_quartiles.csv')

# Step 1: Rename the rainfall shock column
merged_data.rename(columns={'SD_dec_may_1': 'rain'}, inplace=True)

# Step 2: Keep only the relevant columns
# Using regex to match columns that start with 'dum1', 'dum2', 'dum1_p', 'dum2_p'
relevant_columns = ['id', 'rain', 'followup'] + \
                   [col for col in merged_data.columns if col.startswith('dum1') or col.startswith('dum2')]

# Filter the DataFrame to keep only the relevant columns
final_rain_data = merged_data[relevant_columns]

# Step 3: Sort the DataFrame by 'id' and 'followup'
final_rain_data = final_rain_data.sort_values(by=['id', 'followup'])

# Step 4: Save the final DataFrame to a CSV file
final_rain_data.to_csv('/content/drive/My Drive/Rep/Test/Rainshock_data.csv', index=False)

print("Rainshock data prepared and saved as 'Rainshock_data.csv'.")


Rainshock data prepared and saved as 'Rainshock_data.csv'.


In [17]:
import os

# Load the panel_temp data and Rainshock_data
panel_temp = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp.csv')
rainshock_data = pd.read_csv('/content/drive/My Drive/Rep/Test/Rainshock_data.csv')

# Step 1: Sort both datasets by 'id' and 'followup'
panel_temp = panel_temp.sort_values(by=['id', 'followup'])
rainshock_data = rainshock_data.sort_values(by=['id', 'followup'])

# Step 2: Merge the datasets on 'id' and 'followup'
merged_data = pd.merge(panel_temp, rainshock_data, on=['id', 'followup'], how='outer', indicator=True)

# Step 3: Display the merge result
print(merged_data['_merge'].value_counts())

# Step 4: Filter rows where either 'dabi' == 1 or 'progoti' == 1
filtered_data = merged_data[(merged_data['dabi'] == 1) | (merged_data['progoti'] == 1)]

# Step 5: Drop the '_merge' column
filtered_data.drop(columns=['_merge'], inplace=True)

# Step 6: Save the final dataset
filtered_data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_updated.csv', index=False)

# Step 7: Delete the temporary file (equivalent to Stata's erase command)
os.remove('/content/drive/My Drive/Rep/Test/panel_temp.csv')

print("Merged data processed and saved as 'panel_temp_updated.csv'. Temporary file 'panel_temp.csv' has been deleted.")


_merge
both          7951
left_only        0
right_only       0
Name: count, dtype: int64
Merged data processed and saved as 'panel_temp_updated.csv'. Temporary file 'panel_temp.csv' has been deleted.


In [18]:
# Assume that `filtered_data` is the merged dataset after previous operations
# If it's not already loaded, load the dataset
filtered_data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_updated.csv')

# Step 1: Generate the attrition indicator for midline (followup == 1)
filtered_data['x'] = 1 - filtered_data['int_done']
filtered_data['x'] = filtered_data['x'].where(filtered_data['followup'] == 1, other=np.nan)

# Group by 'id' and find the maximum value of 'x' (equivalent to `egen attrition = max(x)` in Stata)
filtered_data['attrition'] = filtered_data.groupby('id')['x'].transform('max')

# Drop the temporary 'x' column
filtered_data.drop(columns=['x'], inplace=True)

# Replace NaN values with 1 (equivalent to `replace attrition = 1 if attrition == .` in Stata)
filtered_data['attrition'].fillna(1, inplace=True)

# Label for 'attrition' (in comments, since pandas does not support labels)
# attrition = 1 if not interviewed at midline

# Step 2: Generate the attrition_end indicator for endline (followup == 2)
filtered_data['x'] = 1 - filtered_data['int_done']
filtered_data['x'] = filtered_data['x'].where(filtered_data['followup'] == 2, other=np.nan)

# Group by 'id' and find the maximum value of 'x' for endline
filtered_data['attrition_end'] = filtered_data.groupby('id')['x'].transform('max')

# Drop the temporary 'x' column
filtered_data.drop(columns=['x'], inplace=True)

# Replace NaN values with 1 (equivalent to `replace attrition_end = 1 if attrition_end == .` in Stata)
filtered_data['attrition_end'].fillna(1, inplace=True)

# Label for 'attrition_end' (in comments, since pandas does not support labels)
# attrition_end = 1 if not interviewed at endline

# Save the updated dataset
filtered_data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_updated_with_attrition.csv', index=False)

print("Attrition indicators added and saved as 'panel_temp_updated_with_attrition.csv'.")


Attrition indicators added and saved as 'panel_temp_updated_with_attrition.csv'.


In [19]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_updated_with_attrition.csv')

# Define the list of variables to be adjusted
variables = [
    'BRACLoanValue', 'FurnitureValue', 'InventoryValue', 'MachinesValue', 'OtherAssetValue',
    'ProfitPm', 'Profits', 'RevBestM', 'RevWorstM', 'Revenues', 'SiteValue', 'ToolsValue',
    'VehiclesValue', 'ave_furniture_val', 'ave_machine_val', 'ave_tool_val', 'ave_vehicle_val',
    'cost_electricity', 'cost_fuel', 'cost_rent_mach', 'cost_rent_sites', 'cost_repairs',
    'cost_stock', 'cost_trans', 'cost_wages', 'exp_ceremony', 'exp_clothes', 'exp_cosmetic',
    'exp_dowry', 'exp_educ', 'exp_electricity', 'exp_entertain', 'exp_food', 'exp_fuel',
    'exp_furniture', 'exp_maid', 'exp_other1', 'exp_other2', 'exp_other3', 'exp_trans',
    'exp_utensils', 'hhasset_bike', 'hhasset_car', 'hhasset_cell', 'hhasset_fan', 'hhasset_fridge',
    'hhasset_jewel', 'hhasset_motor', 'hhasset_oth1', 'hhasset_oth2', 'hhasset_oth3', 'hhasset_radio',
    'hhasset_tv', 'land_rent_receive', 'wage_inc_hh', 'wage_inc_resp', 'profit_oth_bus',
    'loan_given1', 'loan_given2', 'loan_given4', 'loan_given5', 'loan_given6', 'loan_given7',
    'loan_given8', 'loan_given9', 'loan_given10', 'loan_given11', 'loan_given12', 'loan_given13',
    'trans_given1', 'trans_given2', 'trans_given3', 'trans_given4', 'trans_given5',
    'trans_received1', 'trans_received2', 'trans_received3', 'NbracloanV1', 'NbracloanV2',
    'NbracloanV3', 'NbracloanV4', 'NbracloanV5', 'NbracloanV6', 'NbracloanV7', 'NbracloanV8',
    'NbracloanV9', 'NbracloanV10', 'NbracloanV11', 'NbracloanV12'
]

# Constants
inflation_rate = 0.0553
usd_ppp_conversion = 28.25

# Step 1: Deflate the midline values (followup == 1) and endline values (followup == 2)
for var in variables:
    # Deflate the midline values
    data.loc[data['followup'] == 1, var] = data.loc[data['followup'] == 1, var] / (1 + inflation_rate)

    # Deflate the endline values
    data.loc[data['followup'] == 2, var] = data.loc[data['followup'] == 2, var] / ((1 + inflation_rate) ** 2)

    # Step 2: Convert to USD PPP
    data[var] = data[var] / usd_ppp_conversion

# Save the updated DataFrame
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_adjusted_to_USD_PPP.csv', index=False)

print("Monetary variables adjusted for inflation and converted to USD PPP. Data saved as 'panel_temp_adjusted_to_USD_PPP.csv'.")


Monetary variables adjusted for inflation and converted to USD PPP. Data saved as 'panel_temp_adjusted_to_USD_PPP.csv'.


In [20]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_adjusted_to_USD_PPP.csv')

# Step 1: Calculate the total value of business assets
# Using the row-wise sum across the specified columns
data['BusinessAssetsValue'] = data[['InventoryValue', 'MachinesValue', 'OtherAssetValue',
                                    'FurnitureValue', 'ToolsValue', 'VehiclesValue']].sum(axis=1, skipna=False)

# Label the variable (in comments, since pandas does not support variable labels)
# BusinessAssetsValue = "Value of business assets"

# Step 2: Generate summary statistics (equivalent to `su` in Stata)
print("\nSummary statistics for BusinessAssetsValue (Dabi = 1):")
print(data[data['dabi'] == 1]['BusinessAssetsValue'].describe())

print("\nSummary statistics for BusinessAssetsValue (Progoti = 1):")
print(data[data['progoti'] == 1]['BusinessAssetsValue'].describe())

print("\nSummary statistics for BusinessAssetsValue (Census = 1):")
print(data[data['census'] == 1]['BusinessAssetsValue'].describe())

# Further breakdown by followup status
print("\nSummary statistics for BusinessAssetsValue (Dabi = 1 & followup = 0):")
print(data[(data['dabi'] == 1) & (data['followup'] == 0)]['BusinessAssetsValue'].describe())

print("\nSummary statistics for BusinessAssetsValue (Dabi = 1 & followup = 1):")
print(data[(data['dabi'] == 1) & (data['followup'] == 1)]['BusinessAssetsValue'].describe())

print("\nSummary statistics for BusinessAssetsValue (Dabi = 1 & followup = 2):")
print(data[(data['dabi'] == 1) & (data['followup'] == 2)]['BusinessAssetsValue'].describe())

# Step 3: Handle outliers
# Replacing the outlier with NaN if BusinessAssetsValue > 20,000,000
data.loc[data['BusinessAssetsValue'] > 20000000, 'BusinessAssetsValue'] = np.nan

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_BusinessAssetsValue.csv', index=False)

print("Business assets value calculated and saved as 'panel_temp_with_BusinessAssetsValue.csv'.")



Summary statistics for BusinessAssetsValue (Dabi = 1):
count    3.202000e+03
mean     1.136979e+04
std      3.763564e+05
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      3.191947e+03
max      2.125434e+07
Name: BusinessAssetsValue, dtype: float64

Summary statistics for BusinessAssetsValue (Progoti = 1):
count    4.456000e+03
mean     2.414987e+04
std      6.443022e+04
min      0.000000e+00
25%      2.208407e+03
50%      9.559292e+03
75%      2.409911e+04
max      1.672500e+06
Name: BusinessAssetsValue, dtype: float64

Summary statistics for BusinessAssetsValue (Census = 1):
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: BusinessAssetsValue, dtype: float64

Summary statistics for BusinessAssetsValue (Dabi = 1 & followup = 0):
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: BusinessAssetsValue, dtype: float64

Summary statistics for Busi

In [21]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_BusinessAssetsValue.csv')

# Step 1: Calculate BusinessHours
data['BusinessHours'] = data['tot_no_hours'] * data['tot_no_days'] * data['tot_no_months']

# Label the variable (in comments, since pandas does not support variable labels)
# BusinessHours = "total # hours per year enterprise operates"

# Step 2: Calculate Business Costs
data['Costs'] = data[['cost_wages', 'cost_repairs', 'cost_rent_sites', 'cost_rent_mach',
                      'cost_trans', 'cost_electricity', 'cost_fuel', 'cost_stock']].sum(axis=1, skipna=False)

# Label the variable (in comments, since pandas does not support variable labels)
# Costs = "Total business costs over the last 12 months"

# Step 3: Handle missing values for Costs
data.loc[data['HasBusiness'].isna(), 'Costs'] = np.nan

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_BusinessHours_and_Costs.csv', index=False)

print("Business hours and costs calculated and saved as 'panel_temp_with_BusinessHours_and_Costs.csv'.")


Business hours and costs calculated and saved as 'panel_temp_with_BusinessHours_and_Costs.csv'.


In [22]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_BusinessHours_and_Costs.csv')

# Step 1: Calculate Esale2yr
data['Esale2yr'] = ((40 * data['prob2_inc20']) + (10 * data['prob2_inc10']) +
                    (0 * data['prob2_nochange']) + (-10 * data['prob2_dec10']) +
                    (-40 * data['prob2_dec20'])) / 100

# Step 2: Calculate SDsale2yr
data['SDsale2yr'] = np.sqrt(
    ((1600 * data['prob2_inc20']) + (100 * data['prob2_inc10']) +
     (0 * data['prob2_nochange']) + (100 * data['prob2_dec10']) +
     (1600 * data['prob2_dec20'])) / 100 - (data['Esale2yr'] ** 2)
)

# Step 3: Calculate CVsale2yr
data['CVsale2yr'] = data['SDsale2yr'] / data['Esale2yr'].abs()

# Label for CVsale2yr (in comments, as pandas does not support variable labels)
# CVsale2yr = "Coefficient of variation of expected sales growth in 2 years"

# Step 4: Tagging and calculating mean CVsale2yr by branch and followup for census data
data['tag'] = data.groupby(['identifier_branch', 'followup'])['identifier_branch'].transform('size') == 1

data['MCVsale2yr'] = data.groupby(['identifier_branch', 'followup'])['CVsale2yr'].transform('mean').where(data['census'] == 1)

# Step 5: Calculate xCVsale2yr for baseline (followup == 0)
data['xCVsale2yr'] = data['MCVsale2yr'].where(data['followup'] == 0)

# Step 6: Calculate the maximum BMCVsale2yr by branch
data['BMCVsale2yr'] = data.groupby('identifier_branch')['xCVsale2yr'].transform('max')

# Step 7: Calculate the median of BMCVsale2yr for tagging HBMCVsale2yr
BMCVsale2yr_median = data.loc[(data['tag'] == 1) & (data['followup'] == 0), 'BMCVsale2yr'].median()

data['HBMCVsale2yr'] = data['BMCVsale2yr'] >= BMCVsale2yr_median

# Label for HBMCVsale2yr (in comments, as pandas does not support variable labels)
# HBMCVsale2yr = "High expected demand uncertainty"

# Step 8: Drop intermediate variables
data.drop(columns=['tag', 'Esale2yr', 'SDsale2yr', 'CVsale2yr', 'BMCVsale2yr', 'MCVsale2yr', 'xCVsale2yr'], inplace=True)

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_HBMCVsale2yr.csv', index=False)

print("Expected sales growth uncertainty calculated and saved as 'panel_temp_with_HBMCVsale2yr.csv'.")


Expected sales growth uncertainty calculated and saved as 'panel_temp_with_HBMCVsale2yr.csv'.


In [23]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_HBMCVsale2yr.csv')

# Step 1: Calculate total household income
data['HHIncome'] = data[['wage_inc_hh', 'wage_inc_resp', 'profit_oth_bus', 'Profits']].sum(axis=1, skipna=False)

# Label for HHIncome (in comments, as pandas does not support variable labels)
# HHIncome = "Household income"

# Step 2: Calculate owner's hours worked in the main household business
data['HoursWorkedBuss'] = data['owner_months'] * data['owner_days_pm'] * data['owner_hrs_pd']
data['HoursWorkedBuss'] = data['HoursWorkedBuss'].where(data['HasBusiness'] != 0, 0)

# Label for HoursWorkedBuss (in comments)
# HoursWorkedBuss = "Owner's hours worked"

# Step 3: Calculate the value of Non-BRAC borrowing
# NbracloanV? matches columns like 'NbracloanV1', 'NbracloanV2', etc.
# Use regex to select columns with 'NbracloanV' prefix
nbracloanv_columns = data.filter(regex=r'^NbracloanV\d+').columns
data['NonBRACLoanValue'] = data[nbracloanv_columns].sum(axis=1, skipna=False)

# Label for NonBRACLoanValue (in comments)
# NonBRACLoanValue = "Non-BRAC loan value"

# Step 4: Determine if any Non-BRAC loan exists
data['NonBRACLoanD'] = data['NonBRACLoanValue'] > 0
data['NonBRACLoanD'] = data['NonBRACLoanD'].where(data['NonBRACLoanValue'].notna(), np.nan)

# Label for NonBRACLoanD (in comments)
# NonBRACLoanD = "Any Non-BRAC loan"

# Step 5: Calculate the total value of transfers or loans given
# loan_given? matches columns like 'loan_given1', 'loan_given2', etc.
# trans_given? matches columns like 'trans_given1', 'trans_given2', etc.
trlendv_columns = data.filter(regex=r'^(loan_given|trans_given)\d*').columns
data['TrlendV'] = data[trlendv_columns].sum(axis=1, skipna=False)

# Label for TrlendV (in comments)
# TrlendV = "Transfers or loans given"

# Step 6: Calculate the total value of transfers received
# trans_received? matches columns like 'trans_received1', 'trans_received2', etc.
trans_received_columns = data.filter(regex=r'^trans_received\d*').columns
data['TransfersReceived'] = data[trans_received_columns].sum(axis=1, skipna=False)

# Label for TransfersReceived (in comments)
# TransfersReceived = "Transfers received"

# Step 7: Calculate net borrowing and transfers
data['LoanTrNet'] = (data['BRACLoanValue'] + data['NonBRACLoanValue'] +
                     data['TransfersReceived'] - data['TrlendV'])

# Label for LoanTrNet (in comments)
# LoanTrNet = "Net borrowing and transfers"

# Step 8: Calculate total land size
data['LandSize'] = data[['landS_cult', 'landS_mortgage', 'landS_other', 'landS_rentout', 'landS_sharecrop']].sum(axis=1, skipna=False)

# Label for LandSize (in comments)
# LandSize = "Size of land owned (decimals)"

# Step 9: Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_final.csv', index=False)

print("All variables calculated and saved as 'panel_temp_final.csv'.")


All variables calculated and saved as 'panel_temp_final.csv'.


In [24]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_final.csv')

# Step 1: Calculate the total value of non-business household assets
data['NonBusinessAssets'] = data[['hhasset_radio', 'hhasset_tv', 'hhasset_fan', 'hhasset_fridge',
                                  'hhasset_cell', 'hhasset_bike', 'hhasset_motor', 'hhasset_jewel',
                                  'hhasset_car', 'hhasset_oth1', 'hhasset_oth2', 'hhasset_oth3']].sum(axis=1, skipna=False)

# Label for NonBusinessAssets (in comments, as pandas does not support variable labels)
# NonBusinessAssets = "Value of non-business household assets"

# Step 2: Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_NonBusinessAssets.csv', index=False)

print("Non-business household assets calculated and saved as 'panel_temp_with_NonBusinessAssets.csv'.")


Non-business household assets calculated and saved as 'panel_temp_with_NonBusinessAssets.csv'.


In [25]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_NonBusinessAssets.csv')

# Step 1: Replace negative food expenditure values with NaN
data['exp_food'] = data['exp_food'].where(data['exp_food'] >= 0, other=np.nan)

# Step 2: Calculate yearly food expenditure adjusted for Ramadan
data['exp_food_year'] = data['exp_food'] * 48  # Adjusting for Ramadan

# Step 3: Calculate temporary expenditure (monthly expenses multiplied by 12)
data['tempexp'] = (data['exp_fuel'] * 12 + data['exp_electricity'] * 12 + data['exp_trans'] * 12 +
                   data['exp_cosmetic'] * 12 + data['exp_maid'] * 12 + data['exp_entertain'] * 12)

# Step 4: Calculate total expenditure
# 'exp_other?' matches columns like 'exp_other1', 'exp_other2', etc.
exp_other_columns = data.filter(regex=r'^exp_other\d*').columns
data['total_expenditure'] = data[['exp_food_year', 'tempexp', 'exp_clothes', 'exp_utensils',
                                  'exp_furniture', 'exp_ceremony', 'exp_dowry', 'exp_educ'] + list(exp_other_columns)].sum(axis=1, skipna=False)

# Handle cases where total_expenditure is zero
data['total_expenditure'] = data['total_expenditure'].where(data['total_expenditure'] != 0, other=np.nan)

# Label for total_expenditure (in comments, as pandas does not support variable labels)
# total_expenditure = "total expenditure last year"

# Step 5: Calculate per-capita expenditure (PCE)
data['PCE'] = data['total_expenditure'] / data['HHsize']

# Label for PCE (in comments)
# PCE = "Consumption per-capita"

# Step 6: Drop intermediate columns
data.drop(columns=['exp_food_year', 'tempexp'], inplace=True)

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_PCE.csv', index=False)

print("Per-capita expenditure calculated and saved as 'panel_temp_with_PCE.csv'.")


Per-capita expenditure calculated and saved as 'panel_temp_with_PCE.csv'.


In [26]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_PCE.csv')

# Step 1: Conditionally replace `RevBestM` and `RevWorstM` based on `Rev_vary`
data['RevBestM'] = data['RevBestM'].where(data['Rev_vary'] != 0, other=data['Revenues'] / 12)
data['RevWorstM'] = data['RevWorstM'].where(data['Rev_vary'] != 0, other=data['Revenues'] / 12)

# Step 2: Conditionally replace `RevBestM` and `RevWorstM` if `HasBusiness` is 0
data['RevBestM'] = data['RevBestM'].where(data['HasBusiness'] != 0, other=0)
data['RevWorstM'] = data['RevWorstM'].where(data['HasBusiness'] != 0, other=0)

# Step 3: Calculate the range of monthly revenues
data['RevRange'] = data['RevBestM'] - data['RevWorstM']

# Label for RevRange (in comments, as pandas does not support variable labels)
# RevRange = "Range of monthly revenues"

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_RevRange.csv', index=False)

print("Range of monthly revenues calculated and saved as 'panel_temp_with_RevRange.csv'.")


Range of monthly revenues calculated and saved as 'panel_temp_with_RevRange.csv'.


In [27]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_RevRange.csv')

# Step 1: Initialize riskaversion to 0 where risk_choice1 == 2
data['riskaversion'] = np.where(data['risk_choice1'] == 2, 0, np.nan)

# Step 2: Replace riskaversion based on conditions
data['riskaversion'] = np.where((data['riskaversion'].isna()) & (data['risk_choice2'] == 2), 1, data['riskaversion'])
data['riskaversion'] = np.where((data['riskaversion'].isna()) & (data['risk_choice3'] == 2), 2, data['riskaversion'])
data['riskaversion'] = np.where((data['riskaversion'].isna()) & (data['risk_choice4'] == 2), 3, data['riskaversion'])
data['riskaversion'] = np.where((data['riskaversion'].isna()) & (data['risk_choice5'] == 2), 4, data['riskaversion'])
data['riskaversion'] = np.where((data['riskaversion'].isna()) & (data['risk_choice5'] == 1), 5, data['riskaversion'])

# Label for riskaversion (in comments, as pandas does not support variable labels)
# riskaversion = "risk aversion (0=low, 5=high)"

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_riskaversion.csv', index=False)

print("Risk aversion calculated and saved as 'panel_temp_with_riskaversion.csv'.")


Risk aversion calculated and saved as 'panel_temp_with_riskaversion.csv'.


In [28]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_riskaversion.csv')

# Step 1: Initialize patience to NaN (equivalent to Stata's missing value '.')
data['patience'] = np.nan

# Step 2: Replace patience based on the conditions provided
data['patience'] = np.where(data['time_pref6'] == 1, 1, data['patience'])
data['patience'] = np.where((data['patience'].isna()) & (data['time_pref6'] == 2) & (data['time_pref5'] == 1), 2, data['patience'])
data['patience'] = np.where((data['patience'].isna()) & (data['time_pref6'] == 2) & (data['time_pref5'] == 2) & (data['time_pref4'] == 1), 3, data['patience'])
data['patience'] = np.where((data['patience'].isna()) & (data['time_pref6'] == 2) & (data['time_pref5'] == 2) & (data['time_pref4'] == 2) & (data['time_pref3'] == 1), 4, data['patience'])
data['patience'] = np.where((data['patience'].isna()) & (data['time_pref6'] == 2) & (data['time_pref5'] == 2) & (data['time_pref4'] == 2) & (data['time_pref3'] == 2) & (data['time_pref2'] == 1), 5, data['patience'])
data['patience'] = np.where((data['patience'].isna()) & (data['time_pref6'] == 2) & (data['time_pref5'] == 2) & (data['time_pref4'] == 2) & (data['time_pref3'] == 2) & (data['time_pref2'] == 2) & (data['time_pref1'] == 1), 6, data['patience'])
data['patience'] = np.where((data['patience'].isna()) & (data['time_pref6'] == 2) & (data['time_pref5'] == 2) & (data['time_pref4'] == 2) & (data['time_pref3'] == 2) & (data['time_pref2'] == 2) & (data['time_pref1'] == 2), 7, data['patience'])

# Label for patience (in comments, as pandas does not support variable labels)
# patience = "patience (1=low, 7=high)"

# Save the updated dataset
data.to_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_patience.csv', index=False)

print("Patience variable calculated and saved as 'panel_temp_with_patience.csv'.")


Patience variable calculated and saved as 'panel_temp_with_patience.csv'.


In [29]:
# Load your dataset if it's not already loaded
data = pd.read_csv('/content/drive/My Drive/Rep/Test/panel_temp_with_patience.csv')

# Step 1: Impute baseline values for the specified variables
variables_to_impute = ['agrisec', 'hireLanyE', 'loans_taken', 'retail', 'startbus']

for var in variables_to_impute:
    # Create the imputed baseline variable (b_var)
    data[f'b_{var}'] = data.groupby('id')[var].transform('max')

# Step 2: Label the variables (in comments, since pandas does not support labels)
# b_agrisec = "Sector: Agriculture-related"
# b_hireLanyE = "Wants to hire new worker"
# b_loans_taken = "=1 if at baseline anyone in hh had ever taken out any loan"
# b_retail = "Sector: Retail"
# b_startbus = "Wants to start business"

# Step 3: Sort the DataFrame by 'id' and 'followup'
data = data.sort_values(by=['id', 'followup'])

# Step 4: Reorder columns alphabetically, except for some key variables
# Get the list of columns in alphabetical order
all_columns_sorted = sorted(data.columns)

# Define the key variables that should appear at the top
key_variables = ['id', 'region', 'identifier_branch', 'brac_loanee_vo_no', 'followup', 'endline', 'treatment', 'dabi', 'progoti', 'census']

# Create the final column order
final_column_order = key_variables + [col for col in all_columns_sorted if col not in key_variables]

# Reorder the DataFrame columns
data = data[final_column_order]

# Step 5: Save the dataset to a new file
data.to_csv('/content/drive/My Drive/Rep/Test/Data_for_analysis.csv', index=False)

print("Imputed baseline values, sorted and reordered columns, and saved the dataset as 'Data_for_analysis.csv'.")


Imputed baseline values, sorted and reordered columns, and saved the dataset as 'Data_for_analysis.csv'.
